[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shhommychon/DeZero-Koki/blob/master/from_scratch_3/steps/step14.ipynb)

# 제1고지 미분 자동 계산

In [1]:
import numpy as np


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


In [2]:
# class Exp(Function):
#     def forward(self, x):
#         return np.exp(x)

#     def backward(self, gy):
#         x = self.input.data
#         gx = np.exp(x) * gy
#         return gx


# def exp(x):
#     return Exp()(x)


In [3]:
def numerical_diff(f, x, eps=1e-4):
    """

      param:
        f (Function): 미분의 대상이 되는 함수
        x (Variable): 미분을 계산하는 변수
        eps (float): 작은 값
    """
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

# 제2고지 자연스러운 코드로

## 11단계 가변 길이 인수(순전파 편)

## 12단계 가변 길이 인수(개선 편)

## 13단계 가변 길이 인수(역전파 편)

In [4]:
# class Square(Function):
#     def forward(self, x):
#         return x ** 2

#     def backward(self, gy):
#         x = self.inputs[0].data
#         gx = 2 * x * gy
#         return gx


# def square(x):
#     return Square()(x)


## 14단계 같은 변수 반복 사용

In [5]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ output.grad for output in f.outputs ]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple): gxs = gxs,

            for x, gx in zip(f.inputs, gxs):
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)


In [6]:
class Function:
    """

    * Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
    * 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현합니다.
    """
    def __call__(self, *inputs):
        xs = [ x.data for x in inputs ]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple): ys = ys,
        outputs = [ Variable(as_array(y)) for y in ys ]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [7]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)

In [8]:
x = Variable(np.array(3.0))
y = add(x, x)
print('y', y.data)

y.backward()
print("x.grad", x.grad)

y 6.0
x.grad 1.0


### 14.1 문제의 원인

In [9]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ output.grad for output in f.outputs ]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple): gxs = gxs,

            for x, gx in zip(f.inputs, gxs):
                x.grad = gx  # 여기가 실수!

                if x.creator is not None:
                    funcs.append(x.creator)


### 14.2 해결책

In [10]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ output.grad for output in f.outputs ]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple): gxs = gxs,

            for x, gx in zip(f.inputs, gxs):
                #########################
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                #########################

                if x.creator is not None:
                    funcs.append(x.creator)


In [11]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

2.0


In [12]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print(x.grad)

3.0


### 14.3 미분값 재설정

In [13]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# 두 번째 계산 (같은 x를 사용하여 다른 계산을 수행)
y = add(add(x, x), x)
y.backward()
print(x.grad)  # x.grad는 3이어야 맞음

2.0
5.0


In [14]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ output.grad for output in f.outputs ]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple): gxs = gxs,

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)

    #########################
    def cleargrad(self):
        self.grad = None
    #########################


In [15]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# 두 번째 계산 (같은 x를 사용하여 다른 계산을 수행)
x.cleargrad()  # 미분값 초기화
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
3.0


## Appendix A 인플레이스 연산

### A.1 문제 확인

### A.2 복사와 덮어 쓰기

In [16]:
import numpy as np
x = np.array(1)
id(x)

139857011574032

In [17]:
x += x  # 덮어쓰기 -> 메모리의 값을 직접 덮어 쓰는 "인플레이스 연산"
id(x)

139857011574032

In [18]:
x = x + x  # 복사(새로 생성)
id(x)

139857222876368

### A.3 DeZero의 역전파에서는

In [19]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ output.grad for output in f.outputs ]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple): gxs = gxs,

            for x, gx in zip(f.inputs, gxs):
                #########################
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad += gx
                #########################

                if x.creator is not None:
                    funcs.append(x.creator)


In [20]:
x = Variable(np.array(3))
y = add(x, x)
y.backward()

print(f"y.grad: {y.grad}({id(y.grad)})")  # y.grad는 1이어야 맞음
print(f"x.grad: {x.grad}({id(x.grad)})")

y.grad: 2(139857011578256)
x.grad: 2(139857011578256)


In [21]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [ output.grad for output in f.outputs ]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple): gxs = gxs,

            for x, gx in zip(f.inputs, gxs):
                #########################
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                #########################

                if x.creator is not None:
                    funcs.append(x.creator)


In [22]:
x = Variable(np.array(3))
y = add(x, x)
y.backward()

print(f"y.grad: {y.grad}({id(y.grad)})")
print(f"x.grad: {x.grad}({id(x.grad)})")

y.grad: 1(139857011201328)
x.grad: 2(139857222876144)
